In [21]:
from lark import Discard
from lark import Lark,Token,Tree
from lark.tree import pydot__tree_to_png
from lark.visitors import Interpreter
import datetime

grammar = '''
turmas: turma+
turma : ("TURMA" MAIUSCULAS alunos PONTO)
alunos: aluno (PONTOVIRGULA aluno)*
aluno: NOME "(" notas ")"
notas: NUMERO ( "," NUMERO)*
MAIUSCULAS:("A".."Z")+
NOME: ("A".."Z"|"a".."z")+
NUMERO:"0".."9"+
PONTO:"."
PONTOVIRGULA:";"
%import common.WS
%ignore WS
'''


In [22]:
def query(nome,nota,turma):
  data = datetime.datetime.now()
  
  with open('resultado.sql','a') as f:
    f.write(f"""INSERT INTO Resultado ("nomeAluno", "nota", "dataInsercao", "turma")
                          VALUES  ("{nome}",{nota},{data},"{turma}");\n""")

def trataElementos(elemento,lst,dic_notas,self):
  for alunos in elemento.children:
      if isinstance(alunos,Tree):
          r = self.visit(alunos)
          for aluno in r:
            media = 0
            for x in aluno[1]:
              media += x
              
              query(aluno[0],x,elemento.children[0])

              if str(x) in dic_notas.keys():
                dic_notas[str(x)].update({aluno[0]})
              else:
                dic_notas[str(x)] = {aluno[0]}

            media /= self.num_disciplinas
            lst.append((aluno[0],aluno[1],round(media)))
  return lst

class MyInterpreter(Interpreter):
  count = 0
  dic_turmas = {}
  num_disciplinas = 0
    
  def turmas(self,elemento):
    self.visit_children(elemento)
    return [self.count,self.dic_turmas]

  def turma(self,elemento):
    dic_notas = {}
    lst = []
    lst = trataElementos(elemento,lst,dic_notas,self)
    
    self.dic_turmas[str(elemento.children[0])] = {"turma": lst, "notas": dic_notas, "num_disciplinas": self.num_disciplinas}
    self.num_disciplinas = 0

  def alunos(self,elemento):
    alunos = []
    for aluno in elemento.children:
      if isinstance(aluno,Tree):
        alunos.append(self.visit(aluno))
        
    return alunos
  
  def aluno(self,elemento):
    self.count+=1
    notas = []
    for nota in elemento.children[1].children:
      if isinstance(nota,Token):
        notas.append(int(nota))

    if len(notas) > self.num_disciplinas:
      self.num_disciplinas = len(notas)

    return (str(elemento.children[0]), notas)
    

In [23]:
def markdwon(data,f):
  md = ""
  md += "# Visualizador de turmas\n"
  for turma in data[1].keys():
    alunos_ = []
    md += f"## {turma}\n"
    md += "### Lista de alunos \n"
    for aluno in data[1][turma]["turma"]:
      alunos_.append((aluno[0],aluno[2]))
      md += f"- {aluno[0]}\n"

    md += "### Notas \n\n"
    md += "| Aluno | Media |\n"
    for aluno in alunos_:
      md += f"| {aluno[0]} | {aluno[1]} |\n"
    md += "\n"
  f.write(md)
  f.close()
  

def header_table(len,f,turma):
  html = ""
  html += f"<h1>{turma}<code><br /></code></h1>"
  html += """<table style="border-collapse: collapse; width: 100%; height: 81px;" border="1">
  <tbody>"""
  html += """<tr style="height: 18px;">"""
  html += """<td style="width: 10%; height: 18px; text-align: center;">Nome</td>"""
  for i in range(len):
    html += f"""<td style="width: 10%; height: 18px; text-align: center;">Nota{i+1}</td>"""
  html += """<td style="width: 10%; height: 18px; text-align: center;">M&eacute;dia</td>"""
  html += """<td style="width: 10%; height: 18px; text-align: center;">&nbsp;</td>"""
  html += """</tr>"""
  return html

def html(data,f):
  html = ""
  for turma in data[1].keys():
    html += header_table(data[1][turma]["num_disciplinas"],f,turma)

    for aluno in data[1][turma]["turma"]:
      html += """<tr style="height: 21px;">"""
      html += f"""<td style="width: 10%; text-align: center; height: 21px;">{aluno[0]}</td>"""
      for i in range(data[1][turma]["num_disciplinas"]):
        nota = aluno[1][i] if i < len(aluno[1]) else "-"
        html += f"""<td style="width: 10%; text-align: center; height: 21px;">{nota}</td>"""

      html += f"""<td style="width: 10%; text-align: center; height: 21px;">{aluno[2]}</td>"""

      if aluno[2] > 10:
        html += """<td style="width: 10%; text-align: center; height: 21px;"><img src="https://html-online.com/editor/tiny4_9_11/plugins/emoticons/img/smiley-embarassed.gif" alt="embarassed" /></td>"""
      else:
        html += """<td style="width: 10%; text-align: center; height: 21px;"><img src="https://html-online.com/editor/tiny4_9_11/plugins/emoticons/img/smiley-cry.gif" alt="cry" /></td>"""
      html += """</tr>"""
    html += """</tbody>"""
    html += "</table>"
  f.write(html)
  f.close()

In [24]:
# Definir o texto-fonte (input) a analisar
#frase = "TURMA A ana (12, 13, 15, 12, 13, 15, 14); joao (9,7,3,6,9,12); xico (12,16).TURMA B lucas (12, 13, 15, 12, 13, 15, 14); joana (9,7,3,6,9,12); xica (12,16)."
frase = "TURMA A ana (1,2,3); ze (2,4);rui(12). TURMA PL zeca(2); rita(2,4,6). TURMA EG rosa (10,11,12,13)."


f = open('resultado.sql','w')
f2 = open('resultado.md','w')
f3 = open('resultado.html','w')
f.close()

# Inicializar o objeto lark
p = Lark(grammar,start="turmas")
# Criar a parsing tree
parse_tree = p.parse(frase)
# Utilizar o Interpreter para visitar a parsing tree
data=MyInterpreter().visit(parse_tree)

print(data)
markdwon(data,f2)
html(data,f3)

f2.close()
f3.close()


[6, {'A': {'turma': [('ana', [1, 2, 3], 2), ('ze', [2, 4], 2), ('rui', [12], 4)], 'notas': {'1': {'ana'}, '2': {'ana', 'ze'}, '3': {'ana'}, '4': {'ze'}, '12': {'rui'}}, 'num_disciplinas': 3}, 'PL': {'turma': [('zeca', [2], 1), ('rita', [2, 4, 6], 4)], 'notas': {'2': {'zeca', 'rita'}, '4': {'rita'}, '6': {'rita'}}, 'num_disciplinas': 3}, 'EG': {'turma': [('rosa', [10, 11, 12, 13], 12)], 'notas': {'10': {'rosa'}, '11': {'rosa'}, '12': {'rosa'}, '13': {'rosa'}}, 'num_disciplinas': 4}}]
